In [1]:
import torch
import gym as g
from gym import spaces
from connect4 import *
from envs import ConnectNEnv, AdvancedDiscreteEnv
from networks.architecture import RepresentationNetwork, DynamicsNetwork, PredictionNetwork
import numpy as np

In [2]:
env = ConnectNEnv()
env.single_obs_size

(1, 1, 6, 7)

In [3]:
from architecture.game import Game

game = Game(env, 3)

In [6]:
game.reset()
game.step(action=0)
game.step(action=1)
game.step(action=0)
game.step(action=1)
game.step(action=0)
game.step(action=1)
print(game.terminal())
game.present_game()

False
||---------------------||
||| || || || || || || |||
||---------------------||
||| || || || || || || |||
||---------------------||
||| || || || || || || |||
||---------------------||
|||1||2|| || || || || |||
||---------------------||
|||1||2|| || || || || |||
||---------------------||
|||1||2|| || || || || |||
ooooooooooooooooooooooooo


In [7]:
game.step(action=0)

In [9]:
game.terminal()

True